In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt, nonlinsolve, latex, log, Wild, expand_log, logcombine, evaluate,oo, limit
from IPython.display import display, HTML, Math

In [2]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        elif var_name == 'Delta_b':
            symbol_obj = symbols(var_name, negative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

In [3]:
simp = True

## Target function

we want to minimize the effective price, given a set of exponents for the buying and selling asset each.

In [4]:
# linear
def deltaBySpot_(asset, s,v, b, w, j, e, Delta, a, min):
  f = (s - (v + b) * w) / w
  # if asset == 'b':
  #   f = -f
  if simp:
    f = simplify(f)
  return f

deltaBySpot = {asset: deltaBySpot_(asset, *all_symbols[asset]) for asset in assets}
deltaBySpot['b']

-b_b + s_b/w_b - v_b

In [5]:
deltaBySpot['s']

-b_s + s_s/w_s - v_s

In [6]:
# convex
def spotByExp_(s, v, b, w, j, e, Delta, a, min):
  f = a * ((1 + 1/j) ** e)
  if simp:
    f = simplify(f)
  return f
spotByExp = {asset: spotByExp_(*all_symbols[asset]) for asset in assets}
spotByExp['b']

a_b*((j_b + 1)/j_b)**e_b

In [7]:
# linear(convex) = convex
def deltaByExp_(asset, s, v, b, w, j, e, Delta, a, min):
    f = deltaBySpot[asset].subs(s, spotByExp[asset])
    if simp:
        f = simplify(f)
    return f

deltaByExp = {asset: deltaByExp_(asset, *all_symbols[asset]) for asset in assets}
deltaByExp['b']

a_b*(j_b + 1)**e_b/(j_b**e_b*w_b) - b_b - v_b

In [8]:
# convex
eff = Delta_s / -Delta_b
eff

-Delta_s/Delta_b

In [9]:
# convex (see below)
effByExps = eff.subs({Delta_b: deltaByExp['b'], Delta_s: deltaByExp['s']})
if simp:
    effByExps = simplify(effByExps)
effByExps

-j_b**e_b*w_b*(a_s*(j_s + 1)**e_s - j_s**e_s*w_s*(b_s + v_s))/(j_s**e_s*w_s*(a_b*(j_b + 1)**e_b - j_b**e_b*w_b*(b_b + v_b)))

### convexity of the target function

- deltaBySpot is linear
- spotByExp is convex
- deltaByExp is a composition of the two

=> deltaByExp is convex

- negation flips convexity/concavity
- inversion of a positive function flips convexity/concavity => 1 / deltaByExp_buying is convex
- multiplying two nonnegative functions with different variables, that are each convex and independent of the other's variable, results in a convex function

==> effectivePrice = deltaByExp_selling / deltaByExp_buying is convex. qed


## inequality-constraints

- value in A0 of buying must not exceed that of selling
- the exponents must adhere to their upper (buying) resp. lower (selling) bounds given by our equation
- need to buy and sell minimum amounts
- cannot buy more than the available balance
- cannot sell more than maxSelling
- the spot prices must not exceed maxInteger
- bonus: the total number of multiplications for both exponentiations must not exceed expLimit (TODO)

In [10]:
# convex * convex = convex
a0Buying = spotByExp['s'] * deltaByExp['b']
a0Buying

a_s*((j_s + 1)/j_s)**e_s*(a_b*(j_b + 1)**e_b/(j_b**e_b*w_b) - b_b - v_b)

In [11]:
# convex * convex = convex
a0Selling = spotByExp['b'] * deltaByExp['s']
a0Selling

a_b*((j_b + 1)/j_b)**e_b*(a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) - b_s - v_s)

In [12]:
# convex + convex = convex?
a0Bound = a0Buying + a0Selling # <=! 0
a0Bound

a_b*((j_b + 1)/j_b)**e_b*(a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) - b_s - v_s) + a_s*((j_s + 1)/j_s)**e_s*(a_b*(j_b + 1)**e_b/(j_b**e_b*w_b) - b_b - v_b)

In [13]:
print(a0Bound)

a_b*((j_b + 1)/j_b)**e_b*(a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) - b_s - v_s) + a_s*((j_s + 1)/j_s)**e_s*(a_b*(j_b + 1)**e_b/(j_b**e_b*w_b) - b_b - v_b)


In [14]:
H = hessian(a0Bound, [e_b, e_s])
H

Matrix([
[a_b*((j_b + 1)/j_b)**e_b*(a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) - b_s - v_s)*log((j_b + 1)/j_b)**2 + a_s*((j_s + 1)/j_s)**e_s*(a_b*(j_b + 1)**e_b*log(j_b)**2/(j_b**e_b*w_b) - 2*a_b*(j_b + 1)**e_b*log(j_b)*log(j_b + 1)/(j_b**e_b*w_b) + a_b*(j_b + 1)**e_b*log(j_b + 1)**2/(j_b**e_b*w_b)),   a_b*((j_b + 1)/j_b)**e_b*(-a_s*(j_s + 1)**e_s*log(j_s)/(j_s**e_s*w_s) + a_s*(j_s + 1)**e_s*log(j_s + 1)/(j_s**e_s*w_s))*log((j_b + 1)/j_b) + a_s*((j_s + 1)/j_s)**e_s*(-a_b*(j_b + 1)**e_b*log(j_b)/(j_b**e_b*w_b) + a_b*(j_b + 1)**e_b*log(j_b + 1)/(j_b**e_b*w_b))*log((j_s + 1)/j_s)],
[  a_b*((j_b + 1)/j_b)**e_b*(-a_s*(j_s + 1)**e_s*log(j_s)/(j_s**e_s*w_s) + a_s*(j_s + 1)**e_s*log(j_s + 1)/(j_s**e_s*w_s))*log((j_b + 1)/j_b) + a_s*((j_s + 1)/j_s)**e_s*(-a_b*(j_b + 1)**e_b*log(j_b)/(j_b**e_b*w_b) + a_b*(j_b + 1)**e_b*log(j_b + 1)/(j_b**e_b*w_b))*log((j_s + 1)/j_s), a_b*((j_b + 1)/j_b)**e_b*(a_s*(j_s + 1)**e_s*log(j_s)**2/(j_s**e_s*w_s) - 2*a_s*(j_s + 1)**e_s*log(j_s)*log(j_s + 1)/(j_s**e_s*w_s) + a_s*(

In [15]:
topLeft = H[0, 0]
topLeft

a_b*((j_b + 1)/j_b)**e_b*(a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) - b_s - v_s)*log((j_b + 1)/j_b)**2 + a_s*((j_s + 1)/j_s)**e_s*(a_b*(j_b + 1)**e_b*log(j_b)**2/(j_b**e_b*w_b) - 2*a_b*(j_b + 1)**e_b*log(j_b)*log(j_b + 1)/(j_b**e_b*w_b) + a_b*(j_b + 1)**e_b*log(j_b + 1)**2/(j_b**e_b*w_b))

In [16]:
topLeft2 = -(a_b/w_s)*((j_b + 1)/j_b)**e_b \
  *(a_s*((j_s + 1)/j_s)**e_s - w_s *(b_s + v_s)) \
  *log((j_b + 1)/j_b)**2 \
  + a_s*((j_s + 1)/j_s)**e_s \
  * (a_b / w_b) \
  * -((j_b + 1)/j_b)**e_b \
  * (log(j_b) - log(j_b + 1))**2 
topLeft2

-a_b*a_s*((j_b + 1)/j_b)**e_b*((j_s + 1)/j_s)**e_s*(log(j_b) - log(j_b + 1))**2/w_b - a_b*((j_b + 1)/j_b)**e_b*(a_s*((j_s + 1)/j_s)**e_s - w_s*(b_s + v_s))*log((j_b + 1)/j_b)**2/w_s

In [17]:
Eq(topLeft, topLeft2).simplify()

Eq(a_b*((j_b + 1)/j_b)**e_b*(-a_s*w_s*(j_s*(j_s + 1))**e_s*log(j_b/(j_b + 1))**2 - j_s**e_s*w_b*(a_s*(j_s + 1)**e_s - j_s**e_s*w_s*(b_s + v_s))*log((j_b + 1)/j_b)**2)/(j_s**(2*e_s)*w_b*w_s), a_b*(j_b*(j_b + 1))**e_b*(a_s*w_s*(j_s*(j_s + 1))**e_s*(log(j_b)**2 + log(j_b + 1)**2 - log((j_b + 1)**log(j_b**2))) - j_s**e_s*w_b*(-a_s*(j_s + 1)**e_s + j_s**e_s*w_s*(b_s + v_s))*log((j_b + 1)/j_b)**2)/(j_b**(2*e_b)*j_s**(2*e_s)*w_b*w_s))

-> monotonously decreasing wrt both e_b and e_s

-> we get the minimum by plugging in the maxima and vice versa

In [18]:
e_upper_bound_b = log(w_b * (v_b + b_b) /a_b, 1 + 1/j_b)
e_upper_bound_b

log(w_b*(b_b + v_b)/a_b)/log(1 + 1/j_b)

In [19]:
e_lower_bound_s = log(w_s * (v_s + b_s) /a_s, 1 + 1/j_s)
e_lower_bound_s

log(w_s*(b_s + v_s)/a_s)/log(1 + 1/j_s)

In [20]:
minimum = simplify(topLeft2.subs({e_b: e_upper_bound_b, e_s: oo}))
minimum

-oo

In [21]:
maximum = simplify(topLeft2.subs({e_b: -oo, e_s: e_lower_bound_s}))
maximum

0

topLeft is nonpositive -> a0Bound is not convex

In [22]:
# a0Bound = (1 / (a0Buying - a0Selling - 1)) + 1 # <=! 0
# # a0Bound = simplify(a0Bound)
# a0Bound

In [23]:
if simp:
  a0Bound = simplify(a0Bound)
a0Bound

(-a_b*j_s**e_s*w_b*(j_b*(j_b + 1))**e_b*(-a_s*(j_s + 1)**e_s + j_s**e_s*w_s*(b_s + v_s)) - a_s*j_b**e_b*w_s*(j_s*(j_s + 1))**e_s*(-a_b*(j_b + 1)**e_b + j_b**e_b*w_b*(b_b + v_b)))/(j_b**(2*e_b)*j_s**(2*e_s)*w_b*w_s)

In [24]:
def expBound_(asset, s, v, b, w, j, e, Delta, a, min):
    e_bound = log(w * (v + b) /a, 1 + 1/j) # constant wrt e
    if simp:
        e_bound = simplify(e_bound)
    if asset == 'b':
        f = e - e_bound # <=! 0 (upper bound)
    else:
        f = e_bound - e # <=! 0 (lower bound)
    if simp:
        f = simplify(f)
    return f

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

e_b - log((w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b))/a_b**(1/log((j_b + 1)/j_b)))

In [25]:
expBound['s']

-e_s + log((w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s))/a_s**(1/log((j_s + 1)/j_s)))

In [26]:
max_s, I_max = symbols('max_s I_max', positive=True, integer=True)

def maxSpotBound_(asset, s, v, b, w, j, e, Delta, a, min):
  return spotByExp[asset] - I_max # <=! 0

maxSpotBound = {asset: maxSpotBound_(asset, *all_symbols[asset]) for asset in assets}
maxSpotBound['b']

-I_max + a_b*((j_b + 1)/j_b)**e_b

In [27]:
def minAmntBound_(asset, s, v, b, w, j, e, Delta, a, min):
    if asset == 'b':
        d = -Delta
    else:
        d = Delta
    f = min - d # <=! 0
    f = f.subs(Delta, deltaByExp[asset])
    if simp:
        f = simplify(f)
    return f

minAmntBound = {asset: minAmntBound_(asset, *all_symbols[asset]) for asset in assets}
minAmntBound['b']

a_b*(j_b + 1)**e_b/(j_b**e_b*w_b) - b_b + min_b - v_b

In [28]:
minAmntBound['s']

-a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) + b_s + min_s + v_s

In [29]:
def maxAmntBound_(asset, s, v, b, w, j, e, Delta, a, min):
    if asset == 'b':
        f = -Delta - b_b # <=! 0
    else:
        f = Delta - max_s # <=! 0
    f = f.subs(Delta, deltaByExp[asset])
    if simp:
        f = simplify(f)
    return f

maxAmntBound = {asset: maxAmntBound_(asset, *all_symbols[asset]) for asset in assets}
maxAmntBound['b']

-a_b*(j_b + 1)**e_b/(j_b**e_b*w_b) + v_b

In [30]:
maxAmntBound['s']

a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) - b_s - max_s - v_s

## Lagrangian

In [31]:
# Decision variables
x1, x2 = e_b, e_s

# Constraints
g = [
    expBound['b'],
    expBound['s'],
    maxSpotBound['b'],
    maxSpotBound['s'],
    minAmntBound['b'],
    minAmntBound['s'],
    maxAmntBound['b'],
    maxAmntBound['s'],
    a0Bound,
] 

# Lagrange multipliers for m constraints
lambdas = symbols('lambda_1:{}'.format(len(g)+1), nonnegative=True)

# Objective function
f = effByExps

# Lagrangian
L = f + sum([lambdas[i] * g[i] for i in range(len(g))])
if simp:
    L = simplify(L)

# Differentiate Lagrangian with respect to decision variables
dL_dx1 = L.diff(x1)
dL_dx2 = L.diff(x2)
if simp:
    dL_dx1 = simplify(dL_dx1)
    dL_dx2 = simplify(dL_dx2)

# Equations from stationarity
equations = [Eq(dL_dx1, 0), Eq(dL_dx2, 0)]

# Complementary slackness conditions
for i in range(len(g)):
    equations.append(Eq(lambdas[i] * g[i], 0))

# Solve
solutions = nonlinsolve(equations, [x1, x2] + list(lambdas))
# solutions = solve(equations, [x1, x2] + list(lambdas))
solutions

In [ ]:
len(solutions)

In [ ]:
def display_large(expr):
  latex_code = f"\\large {{ {latex(expr)} }}"
  display(Math(latex_code))
    
variables = [x1, x2] + list(lambdas)

for i, solution in enumerate(solutions):
  print(f"Solution {i+1}:")
  for var, expression in zip(variables, solution):
    if expression != 0:
      display_large(Eq(var, expression))

In [ ]:
sol1, sol2, sol3, sol4, sol5, sol6 = list(solutions)

## first solution

In [ ]:
for sol in sol1:
  print(sol.free_symbols)

In [ ]:
def display_large(expr):
    latex_code = f"\\Large {{ {latex(expr)} }}"
    display(Math(latex_code))

# Define wildcard symbols
i1 = Wild('i1', integer=True)
i2 = Wild('i2', integer=True)
i3 = Wild('i3', integer=True)
n  = Wild('n',  positive=True, integer=True)
r0 = Wild('r0', positive=True, real=True)
r1 = Wild('r1', positive=True, real=True)
r2 = Wild('r2', real=True)
r3 = Wild('r3', real=True)
r4 = Wild('r4', nonzero=True, real=True)
r5 = Wild('r5', nonzero=True, real=True)


# Replacement rules TODO check that the wildcard-conditions are met
replacement_rules = {
    i1**-i3 * i2**i3: (i2/i1)**i3,
    log(r1 ** r2): r2 * log(r1),
}

for j in [j_b, j_s]:
    replacement_rules.update({
        log(j) - log(j+1): log(j / (j+1)),
        (j*(j+1))**r2: j**r2 * (j+1)**r2,
        j**-r2 * r3 * (j+1)**r2: r3 * ((j+1)/j)**r2,
        -2*log(I_max/a_b)/log(j/(j + 1)): 2*log(a_b/I_max)/log(j/(j + 1)),
        # -2*log(a_s/(w_s*(b_s+v_s)))/log(j/(j + 1)): 2*log((w_s*(b_s+v_s))/a_s)/log(j/(j + 1)),
    })


# Verify equivalences TODO check the assumptions of the wildcards are met
for key, value in replacement_rules.items():
    difference = simplify(key - value)
    assert difference.equals(0), f"Rule {key} -> {value} is not equivalent: {difference}"
sol1_simp = {}
variables = [x1, x2] + list(lambdas)

for var, expression in zip(variables, sol1):
    for key, value in replacement_rules.items():
        expression = expression.replace(key, value)
    # expression = simplify(expression)
    sol1_simp.update({var: expression})
 
for var, expression in sol1_simp.items():
    if not var in [x1, x2]:
        for key, value in sol1_simp.items():
            if key in [x1, x2]:
                expression = expression.replace(value, key)
                expression = expression.replace(2 * value, 2 * key)
                expression = expression.replace(-2 * value, -2 * key)
                sol1_simp.update({var: expression})
    expression = Eq(var, expression)
    # print(expression)
    display_large(expression)

## second solution

In [ ]:
for sol in sol2:
  print(sol.free_symbols)

In [ ]:
sol2_simp = {}

for var, expression in zip(variables, sol2):
    for key, value in replacement_rules.items():
        expression = expression.replace(key, value)
    # expression = simplify(expression)
    sol2_simp.update({var: expression})
 
for var, expression in sol2_simp.items():
    if not var in [x1, x2]:
        for key, value in sol2_simp.items():
            if key in [x1, x2]:
                expression = expression.replace(value, key)
                expression = expression.replace(2 * value, 2 * key)
                expression = expression.replace(-2 * value, -2 * key)
                sol2_simp.update({var: expression})
    expression = Eq(var, expression)
    # print(expression)
    display_large(expression)

In [ ]:
Eq(sol1_simp[x2], sol2_simp[x2]).simplify()

In [ ]:
Eq(sol1_simp[lambdas[1]], sol2_simp[lambdas[1]]).simplify()

## third solution

In [ ]:
for sol in sol3:
  print(sol.free_symbols)

In [ ]:
sol3_simp = {}

for var, expression in zip(variables, sol3):
    for key, value in replacement_rules.items():
        expression = expression.replace(key, value)
    # expression = simplify(expression)
    sol3_simp.update({var: expression})
 
for var, expression in sol3_simp.items():
    if not var in [x1, x2]:
        for key, value in sol3_simp.items():
            if key in [x1, x2]:
                expression = expression.replace(value, key)
                expression = expression.replace(2 * value, 2 * key)
                expression = expression.replace(-2 * value, -2 * key)
                sol3_simp.update({var: expression})
    expression = Eq(var, expression)
    # print(expression)
    display_large(expression)

In [ ]:
Eq(sol1_simp[x2], sol3_simp[x2]).simplify() and Eq(sol2_simp[x2], sol3_simp[x2]).simplify()

In [ ]:
Eq(sol1_simp[lambdas[1]], sol3_simp[lambdas[1]]).simplify() and Eq(sol2_simp[lambdas[1]], sol3_simp[lambdas[1]]).simplify()

# Analysis

- the exp_bound_selling constraint is binding in all three solutions, which aligns well with the fact that all three have the same optimum for e_s
- e_b is resp. maximized, minimized, or analog to e_s. The corresponding other binding constraints are 

In [ ]:
optimum1 = f.subs(sol1_simp)
display_large(f)
display_large(optimum1)